In [6]:
# import all packages needed
import pandas as pd
import json
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt 
import os
import sys
import time
import pickle
from sklearn.dummy import DummyClassifier
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from scipy.sparse import csr_matrix
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [2]:
# load the dataset and find the active users
import pandas as pd
import json
from tqdm import tqdm
line_count = len(open("yelp_dataset/review.json").readlines())
user_ids, business_ids, stars, dates = [], [], [], []
with open("yelp_dataset/review.json") as f:
    for line in tqdm(f, total=line_count):
        blob = json.loads(line)
        user_ids += [blob["user_id"]]
        business_ids += [blob["business_id"]]
        stars += [blob["stars"]]
        dates += [blob["date"]]
ratings = pd.DataFrame(
    {"user_id": user_ids, "business_id": business_ids, "rating": stars, "date": dates}
)
user_counts = ratings["user_id"].value_counts()
active_users = user_counts.loc[user_counts >= 5].index.tolist()

100%|██████████| 6685900/6685900 [00:57<00:00, 116531.80it/s]


In [3]:
# select the active users in the dataset
ratings_active = ratings[ratings.user_id.isin(active_users)]

# hold the last rating of each user as the test set, and the other ratings of each user as training set
ratings_group = ratings_active.groupby('user_id', as_index=False)

ratings_byDate = ratings_group.apply(lambda _ratings_group: _ratings_group.sort_values(by=['date']))
ratings_byDate = ratings_byDate.set_index('user_id')
ratings_test = ratings_byDate.groupby('user_id').tail(1)
ratings_train = ratings_byDate.groupby('user_id').apply(lambda _ratings_byDate: _ratings_byDate.iloc[:-1])
ratings_train.reset_index(level=0, inplace=True)
ratings_test.reset_index(level=0, inplace=True)

In [10]:
ratings_train_new = ratings_train[['business_id','user_id','rating']]

In [5]:
ratings_train_new.head()

,business_id,user_id,rating
user_id,,,
---1lKK3aKOuomHnwAkAow,5cbsjFtrntUAeUx51FaFTg,---1lKK3aKOuomHnwAkAow,4.0
---1lKK3aKOuomHnwAkAow,--9e1ONYQuAa-CB_Rrw7Tw,---1lKK3aKOuomHnwAkAow,4.0
---1lKK3aKOuomHnwAkAow,ifEHr-ZnGFSKgJVsywiAFg,---1lKK3aKOuomHnwAkAow,5.0
---1lKK3aKOuomHnwAkAow,kosTPb88O4Q0XGbVbEOGCA,---1lKK3aKOuomHnwAkAow,4.0
---1lKK3aKOuomHnwAkAow,rq5dgoksPHkJwJNQKlGQ7w,---1lKK3aKOuomHnwAkAow,5.0


In [11]:
ratings_train_new=ratings_train_new.drop_duplicates(keep='first')

In [12]:
ratings_train_new = ratings_train_new.sample(n=int(ratings_train_new.shape[0]/100))

In [13]:
#Make a matrix with userIds as columns, movieIds as rows and scoring as entries
ratings_matrix = ratings_train_new.pivot(index = 'business_id', columns ='user_id', values = 'rating').fillna(0)

ValueError: Index contains duplicate entries, cannot reshape